In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#test tensor for flattening/unflattening functions
x = np.array([ [[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]],
              
              [ [10, 11, 12],
                [13, 14, 15],
                [16, 17, 18]],
              
              
             [  [19, 20, 21],
                [22, 23, 24],
                [25, 26, 27]],
              
              [ 
                [28, 29, 30],
                [31, 32, 33],
                [34, 35, 36]]
                             ])
#X_ijk = x[frame][frame_row][frame_col]

In [ ]:
x.shape

(4, 3, 3)

In [ ]:
def rank3_first_flattening(tensor):
    #flattening of a tensor along the first component
    
    tensor_faces = tensor.shape[0]
    face_rows = tensor.shape[1]
    face_columns = tensor.shape[2]
    
    T1 = np.empty(shape = (face_rows, tensor_faces*face_columns))
    
    for row_idx in reversed(range(face_rows)):
        flatten = []
        for face_idx in reversed(range(tensor_faces)):
            for col_idx in range(face_columns):
            
                flatten.append(tensor[face_idx][row_idx][col_idx])
        T1[face_rows -(row_idx+1)] = flatten
    
    return T1

In [ ]:
def rank3_second_flattening(tensor):
    #flattening of a tensor along the second component
    
    tensor_faces = tensor.shape[0]
    face_rows = tensor.shape[1]
    face_columns = tensor.shape[2]
    
    T2 = np.empty(shape = (face_columns, face_rows*tensor_faces))
    
    for col_idx in range(face_columns):
        flatten = []
        for row_idx in reversed(range(face_rows)):
            for face_idx in reversed(range(tensor_faces)):
                flatten.append(tensor[face_idx][row_idx][col_idx])
        
        T2[col_idx] = flatten
        
    return T2

In [ ]:
def rank3_third_flattening(tensor):
    #flattening of a tensor along the third component
    
    tensor_faces = tensor.shape[0]
    face_rows = tensor.shape[1]
    face_columns = tensor.shape[2]
    
    T3 = np.empty(shape = (tensor_faces, face_rows*face_columns))
    
    for face_idx in reversed(range(tensor_faces)):
        flatten = []
        for row_idx in reversed(range(face_rows)):
            for col_idx in range(face_columns):
                flatten.append(tensor[face_idx][row_idx][col_idx])
        
        T3[tensor_faces - (face_idx +1)] = flatten
        
    return T3

In [ ]:
#Rank 3 Tensor flattening
def rank3_tensor_flattening(tensor):
    
    #computing first flattening
    T1 = rank3_first_flattening(tensor)
    
    #computing second flattening
    T2 = rank3_second_flattening(tensor)
    
    #computing third flattening
    T3 = rank3_third_flattening(tensor)
    
    return T1, T2, T3

In [ ]:
def rank3_first_unflattening(flattened_tensor, original_tensor):
    #function that takes T(1) and computes its unflattening to the original tensor dimensions
    
    tensor_faces = original_tensor.shape[0]
    face_rows = original_tensor.shape[1]
    face_columns = original_tensor.shape[2]
    
    M1 = np.empty(shape = original_tensor.shape)
    
    for row_idx in reversed(range(face_rows)):
        for face_idx in range(tensor_faces):
            M1[tensor_faces - (face_idx+1)][face_rows - (row_idx+1)] = flattened_tensor[row_idx][face_idx*face_columns:face_columns*(face_idx+1)]
            
    return M1

In [ ]:
def rank3_second_unflattening(flattened_tensor, original_tensor):
    #function that takes T(2) and computes its unflattening to the original tensor dimensions
    
    tensor_faces = original_tensor.shape[0]
    face_rows = original_tensor.shape[1]
    face_columns = original_tensor.shape[2]
    
    flattened_columns = flattened_tensor.shape[1]
    
    M2 = np.empty(shape = original_tensor.shape)
    
    for face_idx in range(tensor_faces):
        
        for row_idx in range(face_rows):
            
            M2[face_idx][row_idx] = flattened_tensor[:, (flattened_columns-face_idx) - (row_idx*tensor_faces+1)]
    
    return M2

In [ ]:
def rank3_third_unflattening(flattened_tensor, original_tensor):
    #function that takes T(3) and computes its unflattening to the original tensor dimensions
    
    tensor_faces = original_tensor.shape[0]
    face_rows = original_tensor.shape[1]
    face_columns = original_tensor.shape[2]
    
    M3 = np.empty(shape = original_tensor.shape)
    
    for face_idx in reversed(range(tensor_faces)):
        rebuilt_face = []
        for col_idx in reversed(range(face_columns)):
            #print(flattened_tensor[face_idx][col_idx*face_columns:col_idx*face_columns + face_columns])
            rebuilt_face.append(flattened_tensor[face_idx][col_idx*face_columns:col_idx*face_columns + face_columns])
        M3[tensor_faces - (face_idx + 1)] = rebuilt_face
    
    return M3

In [ ]:
def get_core_tensor(tensor, U1, U2, U3):
    #S = T x1 (U1)^t x2 (U2)^t x3 (U3)^t
    #formula taken from: 
    #Berkant Savas, Lars Eldén: Handwritten digit classification using higher order singular value decomposition
    #https://doi.org/10.1016/j.patcog.2006.08.004
    
    #compute transposed u1, u2, u3
    U1t = np.transpose(U1)
    U2t = np.transpose(U2)
    U3t = np.transpose(U3)
    
    #computing the T(1) flattening
    T1 = rank3_first_flattening(tensor)
    
    #computing T x1 (U1)^t which is T1 dot U1
    p1 = np.matmul(U1t, T1)
    
    #the product p1 has to be unflattened
    #it is a product along the first component
    up1 = rank3_first_unflattening(p1, tensor)
    
    #computing the second compontent flattening of p1
    Fup1 = rank3_second_flattening(up1)
    
    #computing p2 = p1 x2 (U2)^t = T x1 (U1)^t x2 (U2)^t
    p2 = np.matmul(U2t, Fup1)
    
    #the product p2 has to be unflattened
    #it is a product along the second component
    up2 = rank3_second_unflattening(p2, up1)
    
    #computing the third compontent flattening of p2
    Fup2 = rank3_third_flattening(up2)
    
    #computing S = p2 x3 (U3)^t
    S = np.matmul(U3t, Fup2)
    
    #S must be unflattened
    #it is a product along the third component
    US = rank3_third_unflattening(S, up2)
    
    #rounding to improve readability of the result
    US = np.around(US, decimals = 2)
    
    return(US)
    
    

In [ ]:
#testing on x of all the functions

In [ ]:
T1, T2, T3 = rank3_tensor_flattening(x)

In [ ]:
#making SVD on each flattening using numpy matrices
u1, s1, vt1 = np.linalg.svd(T1, full_matrices = True)
u2, s2, vt2 = np.linalg.svd(T2, full_matrices = True)
u3, s3, vt3 = np.linalg.svd(T3, full_matrices = True)

In [ ]:
print(u1)

[[-0.64633822  0.64465514  0.40824829]
 [-0.57435051 -0.05877771 -0.81649658]
 [-0.50236281 -0.76221056  0.40824829]]


In [ ]:
print(u2)

[[-0.55326743  0.72610501  0.40824829]
 [-0.57702433  0.01939742 -0.81649658]
 [-0.60078122 -0.68731016  0.40824829]]


In [ ]:
print(u3)

[[-0.7575467   0.35513799 -0.43507823  0.33272652]
 [-0.5463285  -0.03905342  0.82810401 -0.1193472 ]
 [-0.3351103  -0.43324484 -0.35097333 -0.75948517]
 [-0.12389209 -0.82743625 -0.04205245  0.54610584]]


In [ ]:
print(T1)

[[34. 35. 36. 25. 26. 27. 16. 17. 18.  7.  8.  9.]
 [31. 32. 33. 22. 23. 24. 13. 14. 15.  4.  5.  6.]
 [28. 29. 30. 19. 20. 21. 10. 11. 12.  1.  2.  3.]]


In [ ]:
rank3_first_unflattening(T1, x)

array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.]],

       [[10., 11., 12.],
        [13., 14., 15.],
        [16., 17., 18.]],

       [[19., 20., 21.],
        [22., 23., 24.],
        [25., 26., 27.]],

       [[28., 29., 30.],
        [31., 32., 33.],
        [34., 35., 36.]]])

In [ ]:
print(T2)

[[34. 25. 16.  7. 31. 22. 13.  4. 28. 19. 10.  1.]
 [35. 26. 17.  8. 32. 23. 14.  5. 29. 20. 11.  2.]
 [36. 27. 18.  9. 33. 24. 15.  6. 30. 21. 12.  3.]]


In [ ]:
rank3_second_unflattening(T2, x)

array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.]],

       [[10., 11., 12.],
        [13., 14., 15.],
        [16., 17., 18.]],

       [[19., 20., 21.],
        [22., 23., 24.],
        [25., 26., 27.]],

       [[28., 29., 30.],
        [31., 32., 33.],
        [34., 35., 36.]]])

In [ ]:
print(T3)

[[34. 35. 36. 31. 32. 33. 28. 29. 30.]
 [25. 26. 27. 22. 23. 24. 19. 20. 21.]
 [16. 17. 18. 13. 14. 15. 10. 11. 12.]
 [ 7.  8.  9.  4.  5.  6.  1.  2.  3.]]


In [ ]:
rank3_third_unflattening(T3, x)

array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.]],

       [[10., 11., 12.],
        [13., 14., 15.],
        [16., 17., 18.]],

       [[19., 20., 21.],
        [22., 23., 24.],
        [25., 26., 27.]],

       [[28., 29., 30.],
        [31., 32., 33.],
        [34., 35., 36.]]])

In [ ]:
#testing core tensor function
core_tensor = get_core_tensor(x, u1, u2, u3)

In [ ]:
core_tensor

array([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -0.0000e+00, -0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

       [[-0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-0.0000e+00, -0.0000e+00,  0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00]],

       [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 6.9700e+00, -4.7000e-01, -0.0000e+00],
        [ 0.0000e+00, -2.3000e+00,  0.0000e+00]],

       [[ 0.0000e+00, -0.0000e+00,  0.0000e+00],
        [ 1.0000e-02, -4.4000e-01, -0.0000e+00],
        [-1.2709e+02, -3.0000e-02,  0.0000e+00]]])

In [ ]:
#suppose the given tensor is the T1 flattening of T
T1 = np.array([[-5,4,5,2], [-1,4,7,2]])

In [ ]:
T1

array([[-5,  4,  5,  2],
       [-1,  4,  7,  2]])

In [ ]:
#unflattening T1 to obtain the original tensor T
T = rank3_first_unflattening(T1, np.zeros(shape = (2,2,2)))

In [ ]:
T

array([[[ 7.,  2.],
        [ 5.,  2.]],

       [[-1.,  4.],
        [-5.,  4.]]])

In [ ]:
#calculating SVD of T(1), T(2), T(3)
u1, s1, vt1 = np.linalg.svd(rank3_first_flattening(T), full_matrices = True)
u2, s2, vt2 = np.linalg.svd(rank3_second_flattening(T), full_matrices = True)
u3, s3, vt3 = np.linalg.svd(rank3_third_flattening(T), full_matrices = True)
#calculating the core tensor for the HOSVD of T
core_tensor_ex5 = get_core_tensor(T, u1, u2, u3)

In [ ]:
core_tensor_ex5

array([[[ 3.16,  0.  ],
        [-0.  , -6.32]],

       [[-0.  ,  0.  ],
        [-9.49,  0.  ]]])

In [ ]:
#using the tensorly library to check for the correctness of our result
import tensorly as tl
from tensorly.decomposition import tucker
core, factors = tl.decomposition.tucker(T, rank=[2, 2,2])

In [ ]:
np.around(core, decimals = 2)

array([[[-9.49, -0.  ],
        [ 0.  ,  0.  ]],

       [[ 0.  , -6.32],
        [ 3.16, -0.  ]]])

In [ ]:
#Example: 
#Assuming that the provided matrix is the T(2) flattening of the original tensor
T2 = np.array([ [1,2,3,4], [5,6,7,8]])
#unflatten to obtain the original tensor T
T = rank3_second_unflattening(T2, np.empty(shape = (2,2,2)))

In [ ]:
#calculating the first flattening
T1 = rank3_first_flattening(T)

In [ ]:
T1

array([[1., 5., 2., 6.],
       [3., 7., 4., 8.]])

In [ ]:
#calculating the second flattening
T2 = rank3_second_flattening(T)

In [ ]:
T2

array([[1., 2., 3., 4.],
       [5., 6., 7., 8.]])

In [ ]:
#calculating the third flattening
T3 = rank3_third_flattening(T)

In [ ]:
T3

array([[1., 5., 3., 7.],
       [2., 6., 4., 8.]])

In [ ]:
#calculating the core tensor for the HOSVD for T

u1, s1, vt1 = np.linalg.svd(rank3_first_flattening(T), full_matrices = True)
u2, s2, vt2 = np.linalg.svd(rank3_second_flattening(T), full_matrices = True)
u3, s3, vt3 = np.linalg.svd(rank3_third_flattening(T), full_matrices = True)

core_tensor_ex7 = get_core_tensor(T, u1, u2, u3)

In [ ]:
core_tensor_ex7

array([[[ 2.400e-01,  2.000e-01],
        [ 2.000e-02,  5.400e-01]],

       [[ 1.000e-02,  1.120e+00],
        [-1.423e+01,  0.000e+00]]])

In [ ]:
#checking the result with tensorly library
core, factors = tl.decomposition.tucker(T, rank=[2, 2,2])

In [ ]:
np.around(core, decimals = 2)

array([[[-1.423e+01,  0.000e+00],
        [-1.000e-02, -1.120e+00]],

       [[-2.000e-02, -5.400e-01],
        [ 2.400e-01,  2.000e-01]]])